# Vergessen: Die unterschätzte Superkraft

Vergessen ist kein Mangel, sondern eine Voraussetzung für Handlungsfähigkeit, weil es Raum für Neues schafft und Überlastung verhindert. Hermann Ebbinghaus entdeckte in seinen bahnbrechenden Experimenten (1885) die mathematische Struktur des Vergessens durch systematische Selbstversuche mit sinnlosen Silben. Seine empirischen Daten zeigen, dass Gedächtnisverlust über Zeit folgt – diese Daten werden heute häufig mit exponentiellen oder Power-Law-Modellen beschrieben.

**Moderne Vereinfachung** – Die exponentiell abfallende Vergessenskurve:

$$R(t) = e^{-t/S}$$

wobei $R$ die Retention (Behaltensleistung), $t$ die Zeit und $S$ die Gedächtnisstabilität ist. Diese Exponentialfunktion ist eine vereinfachte Darstellung. In der Praxis passen komplexere Modelle wie das **Power Law** $R(t) = (1 + t)^{-\beta}$ oft besser zu empirischen Daten, da sie den langsameren Verfall über längere Zeiträume präziser abbilden.

**Spacing Effect und Gedächtniskonsolidierung:** Moderne Forschung erweiterte diese Erkenntnisse um den Spacing Effect – die Beobachtung, dass zeitlich verteilte Wiederholungen die Gedächtnisstabilität durch synaptische Konsolidierung deutlich erhöhen. Nach der **ACT-R-Theorie** (Adaptive Control of Thought–Rational) behält das Gehirn bevorzugt Informationen, die basierend auf ihrer Abrufhistorie statistisch wahrscheinlich wieder benötigt werden – eine adaptive Optimierungsstrategie.

**Interferenz und Speichereffizienz:** Neue Erinnerungen konkurrieren beim Abruf mit ähnlichen alten (retroaktive Interferenz) – so werden redundante Inhalte seltener abgerufen und praktisch weniger präsent. Dies betrifft vor allem das deklarative Gedächtnis (Fakten und Ereignisse), während prozedurale Gedächtnisinhalte (motorische Fähigkeiten) weniger anfällig sind.

**Relevanz für künstliche Systeme:** Für KI-Agenten ist diese Einsicht entscheidend. Ein System, das jedes Gespräch, jedes Datenfragment und jede Zwischennotiz ohne Selektion ablegt, würde ein chaotisches, überfrachtetes System schaffen, das keine Prioritäten kennt und im entscheidenden Moment mit irrelevanten Details überlastet ist.

Die technische Implementierung adaptiven Vergessens folgt mehreren biologisch inspirierten Strategien: zeitbasiertes Decay, relevanzbasierte Konsolidierung und interferenzgesteuerte Selektion.

---

## Adaptives Vergessen – Neurobiologische Grundlagen und praktische Anwendung

Dieses Notebook vermittelt die neurobiologischen und mathematischen Grundlagen des Vergessens und zeigt deren praktische Anwendung in Versicherungs- und Finanzsystemen.

### Problemstellung und Motivation

Vergessen wird häufig als Schwäche oder Defizit betrachtet. Aus neurobiologischer und informationstheoretischer Perspektive ist Vergessen jedoch eine **essenzielle kognitive Funktion**, die mehrere kritische Aufgaben erfüllt:

**Warum ist Vergessen notwendig?**

1. **Kapazitätsmanagement:** Biologische und künstliche Systeme verfügen über begrenzte Speicher- und Verarbeitungskapazität
2. **Interferenzreduktion:** Veraltete oder irrelevante Informationen können aktuelle Entscheidungsprozesse stören
3. **Generalisierung:** Selektives Vergessen von Details ermöglicht die Extraktion allgemeiner Muster
4. **Energieeffizienz:** Die Aufrechterhaltung synaptischer Verbindungen erfordert kontinuierliche metabolische Investition
5. **Adaptivität:** Systeme müssen sich an veränderte Umgebungen anpassen können

### Praktische Relevanz im Versicherungswesen

Im Versicherungswesen stellt sich die Frage: **Wie lange soll ein Schadensfall die Risikobewertung eines Kunden beeinflussen?**

**Praktische Szenarien:**

- **Kfz-Versicherung:** Ein Kunde hatte vor 5 Jahren einen Unfall, seitdem schadenfrei. Soll dieser alte Schaden noch die Prämie erhöhen?
- **Betrugsdetection:** Betrugsmuster ändern sich kontinuierlich. Alte Muster können neue legitime Verhaltensweisen fälschlicherweise als Betrug klassifizieren
- **Risikoprofil-Aktualisierung:** Kundenprofile müssen aktuell bleiben, aber zu häufiges Löschen verliert wertvolle historische Informationen

Diese Probleme erfordern **adaptive Vergessensstrategien**, die biologisch inspirierte Mechanismen nutzen.

---

## Setup und Imports

In [1]:
from __future__ import annotations

import importlib.util
import sys
import warnings
from pathlib import Path

import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact

warnings.filterwarnings("ignore")

# Notebook-Styling laden
NOTEBOOK_DIR = Path.cwd()
PROJECT_ROOT = (
    NOTEBOOK_DIR.parent
    if (NOTEBOOK_DIR.parent / "notebook_style.py").exists()
    else NOTEBOOK_DIR
)

spec = importlib.util.spec_from_file_location(
    "notebook_style", PROJECT_ROOT / "notebook_style.py"
)
if spec is None or spec.loader is None:
    raise ImportError

nb_style = importlib.util.module_from_spec(spec)
sys.modules["notebook_style"] = nb_style
spec.loader.exec_module(nb_style)

PLOT_COLORS = nb_style.setup_plot_style(
    aliases={
        'retention': 'primary',
        'decay': 'secondary',
        'adaptive': 'accent',
        'threshold': 'quaternary',
    },
    cycle_keys=("primary", "secondary", "accent"),
)

SEED = int(getattr(nb_style, 'SEED', 42))
np.random.seed(SEED)

---

### Adaptives Vergessen – Leaky Integrator

Das **Leaky-Integrator Modell** beschreibt, wie Erinnerungen mit der Zeit verblassen:

$$m_t = (1-\lambda) \cdot m_{t-1} + \lambda \cdot x_t$$

**Intuition:**
- $m_t$ = Gedächtnisspur (Schätzung der aktuellen Rate)
- $\lambda$ = Lernrate (wie schnell wir neue Informationen aufnehmen)
- $x_t$ = Neue Beobachtung (z.B. Schadenfrequenz heute)
- $(1-\lambda)$ = Vergessensrate (wie schnell alte Informationen verblassen)

**Neurobiologisches Mapping:**
- $m_t$ <-> Synapsenstärke (Gedächtnisspur)
- $\lambda$ <-> Lern-/Vergessensrate (neuromodulierbar)
- $\tau \approx 1/\lambda$ für kleine $\lambda$; exakt: $\tau = -1/\ln(1-\lambda)$ <-> Zeitkonstante des Synapsendecay

### Adaptive Lernrate

Statt einer festen Lernrate $\lambda$ verwenden wir eine **adaptive Lernrate** $\lambda_t$, die sich an die Überraschung anpasst:

$$\lambda_t = \text{clip}(\lambda_{\min} + k \cdot \frac{|e_t|}{\sigma_t}, \lambda_{\min}, \lambda_{\max})$$

**Komponenten:**
- $e_t = x_t - m_{t-1}$ = Vorhersagefehler (Überraschung)
- $\sigma_t = \sqrt{m_{t-1}}$ = Poisson-Standardabweichung (Erwartete Variabilität)
- $k$ = Überraschungs-Sensitivität (wie stark Überraschung die Lernrate beeinflusst)
- $\lambda_{\min}, \lambda_{\max}$ = Grenzen der Lernrate

**Neurobiologisches Mapping:**
- $\lambda_t$ <-> Neuromodulation durch Überraschung/Fehler (z.B. Dopamin-Signal)
- Höhere Überraschung -> höhere Plastizität -> schnelleres Lernen
- Dies ist biologisch plausibel und entspricht dem Kalman-Filter-Prinzip

---

### Praktisches Szenario: Kfz-Versicherung

**Szenario:** Eine Versicherung möchte die Schadenfrequenz eines Kunden schätzen, um die Prämie zu berechnen.

**Problem:** 
- Mit fester Lernrate $\lambda$ reagiert das System zu langsam auf wechselnde Schadenmuster (z.B. wenn der Kunde plötzlich mehr Schäden meldet)
- Oder es reagiert zu schnell auf Rauschen (einzelne Ausreißer beeinflussen die Schätzung zu stark)

**Lösung:** Adaptive Lernrate, die mit der Überraschung (Vorhersagefehler) steigt.

**Interpretation:**
- Wenn die beobachtete Schadenfrequenz stark von der Schätzung abweicht -> große Überraschung -> höhere Lernrate
- Wenn die beobachtete Schadenfrequenz der Schätzung entspricht -> kleine Überraschung -> niedrigere Lernrate

---

### Implementierung

In [2]:
def simulate_claims(t=300, regimes=None, seed=42):
    """Simuliert Schaden-Claim-Daten."""
    if regimes is None:
        regimes = [(0, 100, 2.0), (100, 200, 5.0), (200, 300, 3.0)]

    rng = np.random.default_rng(seed)
    lam_true = np.zeros(t)
    for start, end, rate in regimes:
        lam_true[start:end] = rate

    x = rng.poisson(lam_true)
    return x, lam_true


def ema_fixed(x, lam=0.05):
    """Exponential Moving Average mit fester Lernrate."""
    m = np.empty_like(x, dtype=float)
    m[0] = x[0]
    for t in range(1, len(x)):
        m[t] = (1 - lam) * m[t-1] + lam * x[t]
    return m


def ema_adaptive(x, lam_min=0.02, lam_max=0.30, k=0.05):
    """Exponential Moving Average mit adaptiver Lernrate."""
    m = np.empty_like(x, dtype=float)
    lam_t = np.empty_like(x, dtype=float)
    m[0] = x[0]
    lam_t[0] = lam_min
    eps = 1e-6

    for t in range(1, len(x)):
        exp_noise = np.sqrt(max(m[t-1], eps))
        e_norm = abs(x[t] - m[t-1]) / (exp_noise + eps)
        lam = np.clip(lam_min + k * e_norm, lam_min, lam_max)
        lam_t[t] = lam
        m[t] = (1 - lam) * m[t-1] + lam * x[t]

    return m, lam_t

### Interaktives Beispiel: adaptive Lernrate

In [3]:
# Daten einmalig simulieren
x, lam_true = simulate_claims(t=300)

def plot_comparison(x, lam_true, m_fixed, m_adapt, lam_t):
    """Visualisiert Beobachtungen Schätzungen und adaptive Lernrate."""
    T = len(x)
    fig, (ax1, ax2) = plt.subplots(
        2, 1, figsize=(20, 7),
        sharex=True,
        gridspec_kw={'height_ratios': [3, 1]}
    )

    ax1.step(np.arange(T), x, where='mid', color='gray', alpha=0.5,
             label='Beobachtete Schäden/Tag', linewidth=1.5)
    ax1.plot(lam_true, 'k--', lw=2, label='Wahre Rate (unbekannt)', alpha=0.7)
    ax1.plot(m_fixed, color=PLOT_COLORS['retention'], label='EMA fix', lw=2, alpha=0.8)
    ax1.plot(m_adapt, color=PLOT_COLORS['adaptive'], label='EMA adaptiv', lw=2, alpha=0.8)

    ax1.set_ylabel('Schadenrate/Tag', fontsize=11, fontweight='bold')
    ax1.legend(loc='upper left', ncol=2, fontsize=10)
    ax1.grid(True, alpha=0.3)
    ax1.set_title('Kfz-Versicherung: Schadenfrequenz-Schätzung', fontsize=12, fontweight='bold')

    ax2.plot(lam_t, color=PLOT_COLORS['adaptive'], lw=2, label='λ_t (adaptiv)')
    ax2.set_ylabel('Lernrate λ_t', fontsize=11, fontweight='bold')
    ax2.set_xlabel('Tag', fontsize=11, fontweight='bold')
    ax2.set_ylim(0, max(lam_t) * 1.1)
    ax2.grid(True, alpha=0.3)
    ax2.legend(loc='upper left', fontsize=10)

    plt.tight_layout()
    plt.show()
    plt.close(fig)


def update_plot(lam_fixed=0.05, lam_min=0.01, lam_max=0.30, k=0.05):
    m_fixed = ema_fixed(x, lam=lam_fixed)
    m_adapt, lam_t = ema_adaptive(x, lam_min=lam_min, lam_max=lam_max, k=k)
    plot_comparison(x, lam_true, m_fixed, m_adapt, lam_t)


# Interaktive Slider
interact(
    update_plot,
    lam_fixed=widgets.FloatSlider(0.05, min=0.005, max=0.30, step=0.005,
                                   description='λ fix:'),
    lam_min=widgets.FloatSlider(0.01, min=0.001, max=0.20, step=0.001,
                                 description='λ_min:'),
    lam_max=widgets.FloatSlider(0.30, min=0.05, max=0.60, step=0.01,
                                 description='λ_max:'),
    k=widgets.FloatSlider(0.05, min=0.00, max=0.20, step=0.005,
                           description='k (Überraschung):')
)

interactive(children=(FloatSlider(value=0.05, description='λ fix:', max=0.3, min=0.005, step=0.005), FloatSlid…

<function __main__.update_plot(lam_fixed=0.05, lam_min=0.01, lam_max=0.3, k=0.05)>

---

### Interpretation der Ergebnisse

**Was beobachten wir?**

1. **Feste Lernrate (blau):**
   - Mit niedriger $\lambda$ (z.B. 0.01): Langsame Anpassung, glatte Kurve, aber verzögerter Regimewechsel
   - Mit hoher $\lambda$ (z.B. 0.20): Schnelle Anpassung, aber viel Rauschen

2. **Adaptive Lernrate (rot):**
   - Passt sich automatisch an: Niedrig bei stabilen Phasen, hoch bei Regimewechseln
   - Unten siehst du $\lambda_t$ über Zeit: Spitzen bei großen Überraschungen

3. **Praktische Implikation:**
   - Adaptive Lernrate kombiniert die Vorteile beider Welten
   - Sie ist robust gegen Rauschen UND reagiert schnell auf echte Änderungen

**Praxisleitfaden:**
- **Konservativ:** Hohe $\lambda_{\min}$, niedrige $\lambda_{\max}$, niedriges $k$ -> Weniger Überreaktion
- **Aggressiv:** Niedrige $\lambda_{\min}$, hohe $\lambda_{\max}$, hohes $k$ -> Schnellere Anpassung
- **Ausgewogen:** Mittlere Werte (Standard) -> Balance zwischen Stabilität und Responsivität

---

### Neurobiologische Grundlagen

#### Warum funktioniert dieses Modell biologisch?

**1. Leaky Integrator <-> Synapsendecay**
- Synapsen verlieren ihre Stärke über Zeit, wenn sie nicht aktiviert werden (Decay)
- Das $(1-\lambda)$ Term modelliert diesen natürlichen Verfall

**2. Adaptive Lernrate <-> Neuromodulation**
- Das Gehirn moduliert die Plastizität (Lernfähigkeit) basierend auf Überraschung/Fehler
- Neurotransmitter wie Dopamin signalisieren Vorhersagefehler und erhöhen die Plastizität
- Dies entspricht dem biologischen Prinzip: "Große Überraschung -> Schnelleres Lernen"

**3. Poisson-Normalisierung <-> Biologische Plausibilität**
- Für Ereignisraten (wie Schadenfrequenz) ist Poisson-Verteilung realistisch
- Die Normalisierung durch $\sqrt{m_{t-1}}$ entspricht der biologischen Erwartung von Variabilität

**4. Kalman-Filter-Logik**
- Dieses Modell ist ein vereinfachter Kalman-Filter
- Kalman-Filter sind in Neurowissenschaft und ML etabliert als biologisch plausible Modelle

---

#### Zusammenfassung
Das Leaky-Integrator-Modell mit adaptiver Lernrate vereint zentrale neurobiologische Prinzipien – synaptischen Zerfall, neuromodulatorische Anpassung und Kalman-Filter-Dynamik – zu einem konsistenten, praxisorientierten Rechenmodell.